In [5]:
# general imports, basic globals

import io
import sys
import subprocess 
import copy
import pandas as pd
import numpy as np
import pyvista as pv
import galois
import json

# from gfq import *
from graph_viz import *
from graph_classes import *

# Define some colors, for the vertices and edges
RED = '#FF0000'
DK_RED = '#990000'
LT_GREEN = '#44FF44'
DK_GREEN = '#009900'
LT_BLUE = '#0088FF'


In [6]:
# Slimfly graph


# the method set_vertex_list() should be set (the default is just the integers from 0 to num_vert-1)
# the method set_adjacency_matrix() MUST be set (the default is all 0s, so no edges)
# basic layouts and other defaults are set in the Graph class from graph_viz.py

# The basic layout is evenly spaced and clockwise around a circle in order.
# If you want anything fancier than the basic layout, inherit from the layout class and do what you want. 
# See er_graph for an example. Same with colors, line opacity and the like.

# visualize_graph() as below should "just work", and shows the basic layout with all graph edges.
# See er_graph for more parameters for visualize_graph()

class SlimflyGraph(Graph):
    
    def set_vertex_list(self, q):
        # in lex order, as defined in the paper
        # this sets 2q groups of vertices, in 2 sets
        self.q = q
        self.GF = galois.GF(self.q)
        self.w = int(q/4)
        self.delta = q%4
        if self.delta==3:
            self.delta = -1
        num_vert = 2*(q**2)
        vertex_list = []
        for g0 in range(2):
            for g1 in range(q):
                for i in range(q):
                    vertex_list.append(Vertex([g0,g1,i]))
        return vertex_list, num_vert

    def set_adjacency_matrix(self):
        # adjacency_matrix[i,j] is 1 if vertex_list[i].value is orthogonal to vertex_list[j].value and 0 otherwise
        adj_matrix = []
        for i in range(self.num_vertices):
            row = [0]*self.num_vertices
            adj_matrix.append(row)            
        self.set_intragroup_connections(adj_matrix)
        self.set_intergroup_connections(adj_matrix)
        return adj_matrix
        
    def set_intragroup_connections(self, adj_matrix):
        X, X_prime = self.generate_gensets()
        for i in range(self.q):
            for j in range(1,self.q):
                index_j = i*self.q+j
                vert_val_j = self.GF(self.vertex_list[index_j].value[2])
                for k in range(j):
                    index_k = i*self.q+k
                    vert_val_k = self.GF(self.vertex_list[index_k].value[2])
                    if (vert_val_j-vert_val_k) in X:
                        adj_matrix[index_j][index_k] = 1
                        adj_matrix[index_k][index_j] = 1
        for i in range(self.q, 2*self.q):
            for j in range(1,self.q):
                index_j = i*self.q+j
                vert_val_j = self.GF(self.vertex_list[index_j].value[2])
                for k in range(j):
                    index_k = i*self.q+k
                    vert_val_k = self.GF(self.vertex_list[index_k].value[2])
                    if (vert_val_j-vert_val_k) in X_prime:
                        adj_matrix[index_j][index_k] = 1
                        adj_matrix[index_k][index_j] = 1

    def generate_gensets(self):
        prim_elt = self.GF.primitive_elements[0]
        prim_elt_squared = prim_elt*prim_elt
        X = [self.GF(1)]
        X_prime = [prim_elt]
        if q%4==0:
            gen_limit = 2*int(self.q/4)-1
            for i in range(gen_limit):
                X.append(X[i]*prim_elt_squared)
                X_prime.append(X_prime[i]*prim_elt_squared)
        else:
            neg_1 = self.GF(0) - self.GF(1)
            gen_limit = int(round(self.q/4))-1
            for i in range(gen_limit):
                X.append(X[i]*prim_elt_squared)
                X_prime.append(X_prime[i]*prim_elt_squared)
            for i in range(gen_limit+1):
                X.append(neg_1*X[i])
                X_prime.append(neg_1*X_prime[i])
        return X, X_prime
 
    def set_intergroup_connections(self, adj_matrix):
        for x in range(self.q):
            X = self.GF(x)
            for m in range(self.q):
                M = self.GF(m)
                for c in range(self.q):
                    C = self.GF(c)
                    y = json.loads(str(M*X+C))
                    i = self.get_index(Vertex([0,x,y]))
                    j = self.get_index(Vertex([1,m,c]))
                    adj_matrix[i][j] = 1
                    adj_matrix[j][i] = 1
                
    def get_index(self, vertex):
        return (self.q**2)*vertex.value[0] + (self.q)*vertex.value[1] + vertex.value[2]
                     
    def set_layouts(self):
        self.layouts = SF_Layouts(self)
    

In [7]:
class SF_Layouts(Layouts):
    
    def set_layouts_SF(self): 
    # in addition to the clockwise default
        self.add_layout(self.interspersed_layout(), 'interspersed') 
        self.add_layout(self.HS_layout(), 'HS') 
        self.add_layout(self.drawer_layout(), 'cakedrawer') 
        return self.layout_list, self.layout_names
            
    def interspersed_layout(self):
        vpos_list = []
        q = self.graph.q
        for i in range(self.graph.num_vertices):
            index = self.graph.vertex_list[i].value[0] + 2*q*self.graph.vertex_list[i].value[1] + 2*self.graph.vertex_list[i].value[2]
            vpos_list.append(self.graph.vertex_list[index].pos[0])
        return vpos_list

    def HS_layout(self):
        # drawers as in the SF paper
        vpos_list = self.graph.num_vertices*[0,0,0]
        q = self.graph.q
        # the pentagons [0,x,y]
        for i in range(q):
            for j in range(q):
                vpos_list[i*q+j] = copy.deepcopy(self.graph.vertex_list[2*(j*q+i)].pos[0])
        # the stars [1,m,c]
        for i in range(q):
            for j in range(q):
                vpos_list[q**2+i*q+j] = copy.deepcopy(self.graph.vertex_list[(2*(j*q+i)-4)%(2*(q**2))].pos[0])
                for k in range(3):
                    vpos_list[q**2+i*q+j][k] *= np.sqrt(0.25)
        return vpos_list

    def drawer_layout(self):
        # drawers as in the SF paper
        vpos_list = self.graph.num_vertices*[0,0,0]
        q = self.graph.q
        for i in range(q):
            for j in range(q):
                vpos_list[i*q+j] = copy.deepcopy(self.graph.vertex_list[i*2*q+j].pos[0])
        for i in range(q):
            for j in range(q):
                vpos_list[q**2+i*q+j] = copy.deepcopy(self.graph.vertex_list[i*2*q+j].pos[0])
                vpos_list[q**2+i*q+j][2] = 0.5
        return vpos_list


In [9]:
##    these are the prime powers q to be run         ##### 
#####    this algorithm is for ODD PRIME POWERS only    ##### 

# Here are some smaller prime powers
#prime_power_list = [2,3,4,5,7,8,9,11,13,16,17,19,23,25,27,29,31,32,37,41,43,47,49,53,59,61,64]

# These primes are of interest.
# Graphs for these primes support [722,3362,13778,55778] routers that have radixes > [29,61,125,251]. 
# May want to change the opacity on edges and maybe the line width in the viz.
#prime_power_list = [19,41,83,167]  

prime_powers = [49,64]

for q in prime_powers:
    sf_graph = SlimflyGraph(q)
    layout_list, layout_names = sf_graph.layouts.set_layouts_SF()
    if q < 11:
        sf_graph.edges_list.list[0].opacity = 0.4
        for i in range(sf_graph.num_vertices):
            sf_graph.vertex_list[i].size = 20
    if q < 11:
        sf_graph.edges_list.list[0].opacity = 0.2
        for i in range(sf_graph.num_vertices):
            sf_graph.vertex_list[i].size = 10
    elif q < 23:
        sf_graph.edges_list.list[0].opacity = 0.1
        for i in range(sf_graph.num_vertices):
            sf_graph.vertex_list[i].size = 7
    elif q < 28:
        sf_graph.edges_list.list[0].opacity = 0.04
        for i in range(sf_graph.num_vertices):
            sf_graph.vertex_list[i].size = 5
    elif q < 51:
        sf_graph.edges_list.list[0].opacity = 0.01
        for i in range(sf_graph.num_vertices):
            sf_graph.vertex_list[i].size = 2
    else:
        sf_graph.edges_list.list[0].opacity = 0.005
        for i in range(sf_graph.num_vertices):
            sf_graph.vertex_list[i].size = 1

# visualize_graph(graph, layout, edge_displaylist, camera_pos, camera_rot, show_labels, fname, caption)
# layouts: ['default', 'another', 'cakedrawer']
# edgelists: ['all_edges']

#     visualize_graph(sf_graph, 'default', 
#                     ['all_edges'], 
#                      'xy', [0,0,0], False,
#                      "./SF_graphs/SF" + str(q) + "_default", 
#                     'SlimFly('+str(q)+'): SlimFly graph, default layout')
    
#     visualize_graph(sf_graph, 'interspersed', 
#                     ['all_edges'], 
#                      'xy', [0,0,0], False,
#                      "./SF_graphs/SF" + str(q) + "_interspersed", 
#                     'SlimFly('+str(q)+'): SlimFly graph, interspersed layout')
    
    visualize_graph(sf_graph, 'HS', 
                    ['all_edges'], 
                     'xy', [0,0,0], False,
                     "./SF_graphs/SF" + str(q) + "_HS", 
                    'SlimFly('+str(q)+'): SlimFly graph, Hoffman-Singleton-inspired layout')
    
    visualize_graph(sf_graph, 'cakedrawer', 
                    ['all_edges'], 
                     'zx', [0,0,45], False,
                     "./SF_graphs/SF" + str(q) + "_cakedrawer", 
                    'SlimFly('+str(q)+'): SlimFly graph, cake drawer layout')
    


6
SlimFly(49): SlimFly graph, Hoffman-Singleton-inspired layout


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

SlimFly(49): SlimFly graph, cake drawer layout


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

SlimFly(64): SlimFly graph, Hoffman-Singleton-inspired layout


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

SlimFly(64): SlimFly graph, cake drawer layout


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)